In [1]:
from argparse import ArgumentParser
from distutils.command import check
import os
import csv

import mmcv
from os import path as op

from mmcls.apis import inference_model, init_model, show_result_pyplot

In [ ]:
# img_dir="" #图片的路径 
# device='cuda:0'  # cuda:0 or cuda:1,2,3...，指定你的gpu
# #注意使用模型集成时需考虑config和checkpoint为list
# config="" #和training config保持一致，str or list
# checkpoint="" #checkpoint path，模型训练结果的路径 str or list
# csv_result="" #the path to save csv file，用于保存csv结果的路径
# pu_label=False # ture or false，选择是否使用伪标签技巧

In [8]:
img_dir="data/led/test" #图片的路径 
device='cuda:0'  # cuda:0 or cuda:1,2,3...，指定你的gpu
#注意使用模型集成时需考虑config和checkpoint为list
config="configs/xunfei2022led/resnet34_1xb32_led.py" #和training config保持一致，str or list
checkpoint="training/resnet50_1xb32_led/epoch_50.pth" #checkpoint path，模型训练结果的路径 str or list
csv_result="resnet34_1xb32_led.csv" #the path to save csv file，用于保存csv结果的路径
pu_label=False # ture or false，选择是否使用伪标签技巧

In [14]:
# 注意，这里是模型集成的部分代码，假如你要使用模型集成，这里并未完全实现，期待您的实现
if isinstance(config, list) and isinstance(checkpoint, list):  # 这一行代码有误 args.config， args.checkpoint 需要你自行修改
    for config, checkpoint in zip(config, checkpoint):
        models = init_model(config, checkpoint, device=device) 
else:
    # build the model from a config file and a checkpoint file
    models = init_model(config, checkpoint, device=device)    


# write to the csv file
headers = ['image', 'label']
with open(csv_result, 'w+', newline="") as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)

    # test  images
    for img in mmcv.track_iter_progress(os.listdir(img_dir)):
        _img = op.join(img_dir, img)
        results = []
        # 当存在多个模型时
        if isinstance(models, list):
            for model in models:
                result = inference_model(model, _img)
                results.append(result)#模型集成，未完成
        else:
            # 当仅为单模型时
            result = inference_model(models, _img)
            
        # 模型集成，伪标签技术（是比赛中常用的很有效的涨点方式），这里我们用pred_score分数去做(PS: 未完整实现，大家如果感兴趣，可以去实现它们)
        pred_label, pred_score, pred_class = result['pred_label'], result['pred_score'], result['pred_class']  
        # write  rows
        
        # 本次比赛为分类问题：这里我们设定一个足够高的阈值, 选择预测概率最有把握的样本的标签作为真实的标签（例如概率为0.99或者概率未0.01的预测标签
        # 将预测然后将得到的有标注的数据加入原始数据继续进行训练，再预测，一直到达停止条件（例如大部分甚至全部unlabeled的样本都被打上了标签），
        # 此时，我们就把未标注的样本通过这种方式标注出来了
        # 这里阈值可以根据实际情况自行设定
        if pu_label == True:
            if pred_score > 0.9:
                row = [img, pred_label]
                f_csv.writerow(row)
        else:
            row = [img, pred_label]
            # import pdb;pdb.set_trace()
            f_csv.writerow(row)


load checkpoint from local path: training/resnet50_1xb32_led/epoch_50.pth
The model and loaded state dict do not match exactly

size mismatch for backbone.layer1.0.conv1.weight: copying a param with shape torch.Size([64, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
size mismatch for backbone.layer1.1.conv1.weight: copying a param with shape torch.Size([64, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
size mismatch for backbone.layer1.2.conv1.weight: copying a param with shape torch.Size([64, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([64, 64, 3, 3]).
size mismatch for backbone.layer2.0.conv1.weight: copying a param with shape torch.Size([128, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([128, 64, 3, 3]).
size mismatch for backbone.layer2.0.downsample.0.weight: copying a param with shape torch.Size([512, 256, 1, 1]) from checkpoint, the shape in current 